In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import matplotlib.pyplot as plt
import mplleaflet
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json
from pprint import pprint

## Bus Stops

In [3]:
gt_stops = pd.read_csv('google_transit/stops.txt', sep=",")
gt_stops = gt_stops.drop(['stop_code','stop_url', 'stop_desc', 'zone_id', 'stop_timezone', 'wheelchair_boarding'], axis=1)

# print(len(gt_stops))
# print(len(gt_stops.loc[gt_stops['stop_code'].notnull()]))
# print(len(gt_stops.loc[gt_stops['stop_id'].notnull()]))
# gt_routes['stop_id'].unique()
gt_stops

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
0,00000020,Kennedy Rd S n/of First Gulf Blvd,43.673256,-79.718468,0,NaN
1,00000030,Kennedy Rd S at Steeles Ave E,43.675159,-79.721710,0,NaN
2,00000055,Rutherford Rd S n/of Steeles Ave E,43.681385,-79.718147,0,NaN
3,00000060,Rutherford Rd S/of Bramsteele Rd,43.681969,-79.718979,0,NaN
4,00000070,Rutherford Rd S n/of Bramsteele Rd,43.683807,-79.721458,0,NaN
...,...,...,...,...,...,...
2961,place_MPVL,Mount Pleasant Village,43.675423,-79.822311,1,NaN
2962,place_SHER,Sheridan College,43.656124,-79.741013,1,NaN
2963,place_SPTE,Trinity Common Terminal,43.731091,-79.763603,1,NaN
2964,place_SWLP,Sandalwood Loop,43.720985,-79.808502,1,NaN


In [4]:
# gt_routes.loc[gt_routes['route_type'] == 3]
# gt_routes['route_long_name'].unique()

## Bus Routes

In [5]:
gt_routes = pd.read_csv('google_transit/routes.txt', sep=",")
gt_routes = gt_routes.drop(['route_url', 'route_type', 'route_desc'], axis=1) 
gt_routes

,route_id,route_short_name,route_long_name
0,1-274,1,Queen
1,2-274,2,Main
2,3-274,3,McLaughlin
3,4-274,4,Chinguacousy
4,5-274,5,Bovaird
...,...,...,...
68,501-274,501,Zum Queen
69,502-274,502,Zum Main
70,505-274,505,Zum Bovaird
71,511-274,511,Zum Steeles


## Calendar & Stop Times & Trips

In [6]:
gt_calendar = pd.read_csv('google_transit/calendar.txt', sep=",")
gt_calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,200302-MULTI-Weekday-01,1,1,1,1,1,0,0,20200302,20200424
1,200302-MULTI-Weekday-01-0001000,0,0,0,1,0,0,0,20200302,20200424
2,200302-MULTI-Weekday-01-0010000,0,0,1,0,0,0,0,20200302,20200424
3,200302-MULTI-Weekday-01-1000000,1,0,0,0,0,0,0,20200302,20200424
4,200302-MULTI-Saturday-01,0,0,0,0,0,1,0,20200307,20200425
5,200302-MULTI-Sunday-01,0,0,0,0,0,0,1,20200308,20200426
6,200302-MULTI-Holiday1-01,1,1,1,1,1,1,1,20200410,20200410


In [25]:
gt_stop_times = pd.read_csv('google_transit/stop_times.txt', sep=",")
gt_stop_times = gt_stop_times.drop(['timepoint', 'drop_off_type', 'pickup_type'], axis=1)
gt_stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,10445576-200302-MULTI-Weekday-01,05:10:00,05:10:00,45565,1
1,10445576-200302-MULTI-Weekday-01,05:12:00,05:12:00,5260,2
2,10445576-200302-MULTI-Weekday-01,05:13:00,05:13:00,5270,3
3,10445576-200302-MULTI-Weekday-01,05:14:00,05:14:00,5280,4
4,10445576-200302-MULTI-Weekday-01,05:14:00,05:14:00,5290,5
...,...,...,...,...,...
486660,10528257-200302-MULTI-Holiday1-01,16:01:00,16:01:00,51112,18
486661,10528257-200302-MULTI-Holiday1-01,16:03:00,16:03:00,51114,19
486662,10528257-200302-MULTI-Holiday1-01,16:05:00,16:05:00,51116,20
486663,10528257-200302-MULTI-Holiday1-01,16:08:00,16:08:00,18030502,21


In [8]:
gt_trips = pd.read_csv('google_transit/trips.txt', sep=",")
gt_trips = gt_trips.drop(['wheelchair_accessible', 'bikes_allowed'], axis=1)
gt_trips

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,17-274,200302-MULTI-Weekday-01,10445576-200302-MULTI-Weekday-01,17 HOWDEN NORTH,0,710327,170018
1,17-274,200302-MULTI-Weekday-01,10445577-200302-MULTI-Weekday-01,17 HOWDEN NORTH,0,710328,170018
2,17-274,200302-MULTI-Weekday-01,10445578-200302-MULTI-Weekday-01,17 HOWDEN NORTH,0,710327,170018
3,17-274,200302-MULTI-Weekday-01,10445579-200302-MULTI-Weekday-01,17 HOWDEN NORTH,0,710328,170018
4,17-274,200302-MULTI-Weekday-01,10445580-200302-MULTI-Weekday-01,17 HOWDEN NORTH,0,710327,170018
...,...,...,...,...,...,...,...
12723,11-274,200302-MULTI-Holiday1-01,10528241-200302-MULTI-Holiday1-01,11 STEELES WESTBOUND,1,712237,110073
12724,511-274,200302-MULTI-Holiday1-01,10528254-200302-MULTI-Holiday1-01,511 ZUM STEELES EASTBOUND,0,712242,5110017
12725,511-274,200302-MULTI-Holiday1-01,10528255-200302-MULTI-Holiday1-01,511 ZUM STEELES EASTBOUND,0,712239,5110017
12726,511-274,200302-MULTI-Holiday1-01,10528256-200302-MULTI-Holiday1-01,511 ZUM STEELES EASTBOUND,0,712240,5110017


## Create Maps

In [18]:
stop_dict = gt_stops.to_dict(orient='records')
stop_dict[:2]

[{'stop_id': '00000020',
  'stop_name': 'Kennedy Rd S n/of First Gulf Blvd',
  'stop_lat': 43.673256,
  'stop_lon': -79.718468,
  'location_type': 0,
  'parent_station': nan},
 {'stop_id': '00000030',
  'stop_name': 'Kennedy Rd S at Steeles Ave E',
  'stop_lat': 43.675159,
  'stop_lon': -79.72171,
  'location_type': 0,
  'parent_station': nan}]

In [21]:
route_dict = gt_routes.to_dict(orient='records')
route_dict[:2]

[{'route_id': '1-274', 'route_short_name': 1, 'route_long_name': 'Queen'},
 {'route_id': '2-274', 'route_short_name': 2, 'route_long_name': 'Main'}]

In [43]:
trips_dict = gt_trips.to_dict(orient='records')
trips_dict[:2]

[{'route_id': '17-274',
  'service_id': '200302-MULTI-Weekday-01',
  'trip_id': '10445576-200302-MULTI-Weekday-01',
  'trip_headsign': '17 HOWDEN NORTH',
  'direction_id': 0,
  'block_id': 710327,
  'shape_id': 170018},
 {'route_id': '17-274',
  'service_id': '200302-MULTI-Weekday-01',
  'trip_id': '10445577-200302-MULTI-Weekday-01',
  'trip_headsign': '17 HOWDEN NORTH',
  'direction_id': 0,
  'block_id': 710328,
  'shape_id': 170018}]

In [50]:
[trips[''] for trips in trips_dict if trips['route_id'] == '1-274']
# stop_map = [stop['stop_name'] for stop in stop_dict if stop['stop_id'] == '00000020']
# stop_map

[{'route_id': '1-274',
  'service_id': '200302-MULTI-Weekday-01',
  'trip_id': '10445994-200302-MULTI-Weekday-01',
  'trip_headsign': '1 QUEEN WEST',
  'direction_id': 1,
  'block_id': 710402,
  'shape_id': 10211},
 {'route_id': '1-274',
  'service_id': '200302-MULTI-Weekday-01',
  'trip_id': '10446268-200302-MULTI-Weekday-01',
  'trip_headsign': '1 QUEEN EAST TO DOWNTOWN TERMINAL',
  'direction_id': 0,
  'block_id': 710436,
  'shape_id': 10197},
 {'route_id': '1-274',
  'service_id': '200302-MULTI-Weekday-01',
  'trip_id': '10447572-200302-MULTI-Weekday-01',
  'trip_headsign': '1 QUEEN EAST',
  'direction_id': 0,
  'block_id': 710425,
  'shape_id': 10157},
 {'route_id': '1-274',
  'service_id': '200302-MULTI-Weekday-01',
  'trip_id': '10448811-200302-MULTI-Weekday-01',
  'trip_headsign': '1 QUEEN WEST',
  'direction_id': 1,
  'block_id': 710427,
  'shape_id': 10210},
 {'route_id': '1-274',
  'service_id': '200302-MULTI-Weekday-01',
  'trip_id': '10448812-200302-MULTI-Weekday-01',
  't

In [46]:
# gt_trips.loc[gt_trips['route_id'] == '17-274']

In [41]:
print(gt_stops.columns)
print()
print(gt_routes.columns)
print()
print(gt_stop_times.columns)
print()
print(gt_trips.columns)

Index(['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'location_type',
       'parent_station'],
      dtype='object')

Index(['route_id', 'route_short_name', 'route_long_name'], dtype='object')

Index(['trip_id', 'arrival_time', 'departure_time', 'stop_id',
       'stop_sequence'],
      dtype='object')

Index(['route_id', 'service_id', 'trip_id', 'trip_headsign', 'direction_id',
       'block_id', 'shape_id'],
      dtype='object')
